## CHATER 5 텐서보드와 모델 재사용

### 5.2 텐서보드 사용하기

In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('TensorFlow-Tutorials-master/05 - TensorBoard, Saver/data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [2]:
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

In [3]:
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

In [4]:
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

In [5]:
tf.summary.scalar('cost', cost)

<tf.Tensor 'cost:0' shape=() dtype=string>

In [6]:
tf.summary.histogram("Weights", W1)

<tf.Tensor 'Weights:0' shape=() dtype=string>

In [7]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('TensorFlow-Tutorials-master/05 - TensorBoard, Saver/model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from TensorFlow-Tutorials-master/05 - TensorBoard, Saver/model\dnn.ckpt-300


In [8]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('TensorFlow-Tutorials-master/05 - TensorBoard, Saver/logs', sess.graph)

In [9]:
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d, ' % sess.run(global_step), 'Cost: %.3f' % sess.run(cost,feed_dict={X: x_data, Y: y_data}))

Step: 301,  Cost: 0.549
Step: 302,  Cost: 0.549
Step: 303,  Cost: 0.549
Step: 304,  Cost: 0.549
Step: 305,  Cost: 0.549
Step: 306,  Cost: 0.549
Step: 307,  Cost: 0.549
Step: 308,  Cost: 0.549
Step: 309,  Cost: 0.549
Step: 310,  Cost: 0.549
Step: 311,  Cost: 0.549
Step: 312,  Cost: 0.549
Step: 313,  Cost: 0.549
Step: 314,  Cost: 0.549
Step: 315,  Cost: 0.549
Step: 316,  Cost: 0.549
Step: 317,  Cost: 0.549
Step: 318,  Cost: 0.549
Step: 319,  Cost: 0.549
Step: 320,  Cost: 0.549
Step: 321,  Cost: 0.549
Step: 322,  Cost: 0.549
Step: 323,  Cost: 0.549
Step: 324,  Cost: 0.549
Step: 325,  Cost: 0.549
Step: 326,  Cost: 0.549
Step: 327,  Cost: 0.549
Step: 328,  Cost: 0.549
Step: 329,  Cost: 0.549
Step: 330,  Cost: 0.549
Step: 331,  Cost: 0.549
Step: 332,  Cost: 0.549
Step: 333,  Cost: 0.549
Step: 334,  Cost: 0.549
Step: 335,  Cost: 0.549
Step: 336,  Cost: 0.549
Step: 337,  Cost: 0.549
Step: 338,  Cost: 0.549
Step: 339,  Cost: 0.549
Step: 340,  Cost: 0.549
Step: 341,  Cost: 0.549
Step: 342,  Cost

In [10]:
summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
writer.add_summary(summary, global_step=sess.run(global_step))

In [11]:
saver.save(sess, 'TensorFlow-Tutorials-master/05 - TensorBoard, Saver/model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
